In [ ]:
# script to compile calculations from new and old workflow

In [1]:
import os
import re
import glob

import rmgpy.data.kinetics

# import importlib
# importlib.reload(rmgpy.data.kinetics)

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [3]:
# first, get valid kinetics from old workflow
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
kinetics_libs = glob.glob(os.path.join(DFT_DIR, 'kinetics', 'reaction*', 'arkane', 'RMG_libraries'))

# Load the Arkane kinetics
entries = []
for i, lib_path in enumerate(kinetics_libs):
    
    # only use kinetics that were verified with vibrational check
    vib_path = os.path.join(os.path.dirname(lib_path), 'vibrational_analysis_check.txt')
    if not os.path.exists(vib_path):
        continue
    with open(vib_path, 'r') as f:
        check = f.read()
        if check != 'True':
            continue
    
    matches = re.search('reaction_([0-9]{4})', lib_path)
    reaction_index = int(matches[1])
    ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
    ark_kinetics_database.load_libraries(lib_path)
    
    # TODO fix bug related to load_libraries not getting the actual name
    for key in ark_kinetics_database.libraries[''].entries.keys():
        entry = ark_kinetics_database.libraries[''].entries[key]
        entry.index = reaction_index
        entries.append(entry)
        print(f'Adding\t{entry.index}\t{entry}')

Adding	127	CH2O(9) + OH(15) <=> H2O(8) + HCO(19)
Adding	180	CH(3) + H2(13) <=> CH2(23) + H(14)
Adding	247	butane(1) + OH(15) <=> H2O(8) + SC4H9(183)
Adding	288	butane(1) + OH(15) <=> H2O(8) + PC4H9(182)
Adding	290	butane(1) + HO2(16) <=> H2O2(17) + PC4H9(182)
Adding	294	CH2O(9) + PC4H9(182) <=> butane(1) + HCO(19)
Adding	324	S(229) <=> S(225)
Adding	325	S(225) <=> S(244)
Adding	326	S(225) <=> S(238)
Adding	370	CH3CHO(35) + SC4H9(183) <=> butane(1) + CH2CHO(21)
Adding	371	CH3CHO(35) + PC4H9(182) <=> butane(1) + CH2CHO(21)
Adding	585	CH2(23) + C3H6(12) <=> CH3(18) + C3H5-A(94)
Adding	591	CH3CHO(35) + C3H5-A(94) <=> C3H6(12) + CH3CO(20)
Adding	650	HO2(16) + C4H8(748) <=> O2(2) + SC4H9(183)
Adding	655	HO2(16) + C4H8(748) <=> O2(2) + PC4H9(182)
Adding	739	C4H8(188) + CH2CHO(21) <=> CH3CHO(35) + C4H7(190)
Adding	748	C4H8(189) + HCO(19) <=> CH2O(9) + C4H7(190)
Adding	749	C4H8(188) + HCO(19) <=> CH2O(9) + C4H7(190)
Adding	754	C4H8(189) + CH2(23) <=> CH3(18) + C4H7(190)
Adding	894	C3H6(12) + S(

In [5]:
# entries

In [6]:
# next, get kinetics from new workflow (overwrite old if repeates)
DFT_DIR = "/work/westgroup/harris.se/autoscience/reaction_calculator/dft/"
kinetics_libs = glob.glob(os.path.join(DFT_DIR, 'kinetics', 'reaction*', 'arkane', 'RMG_libraries'))

# Load the Arkane kinetics
for i, lib_path in enumerate(kinetics_libs):

    matches = re.search('reaction_([0-9]{4})', lib_path)
    reaction_index = int(matches[1])
    ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
    ark_kinetics_database.load_libraries(lib_path)
    print(lib_path)
    # TODO fix bug related to load_libraries not getting the actual name
    existing_indices = [e.index for e in entries]
    for key in ark_kinetics_database.libraries[''].entries.keys():
        entry = ark_kinetics_database.libraries[''].entries[key]
        entry.index = reaction_index
        
        # overwrite existing
        try:
            place = existing_indices.index(reaction_index)
            entries[place] = entry
            print(f'Overwriting existing entry {reaction_index}')
        except ValueError:
            entries.append(entry)
            print(f'Adding new entry {reaction_index}')
            


/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0213/arkane/RMG_libraries
Adding new entry 213
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0370/arkane/RMG_libraries
Overwriting existing entry 370
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0410/arkane/RMG_libraries
Adding new entry 410
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0419/arkane/RMG_libraries
Adding new entry 419
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0420/arkane/RMG_libraries
Adding new entry 420
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0422/arkane/RMG_libraries
Adding new entry 422
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0459/arkane/RMG_libraries
Adding new entry 459
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0464/arkane/RMG_libra

In [ ]:
kinetics_libs

In [ ]:
top_50 = [  # for 1 week RMG
931,213,915,2422,748,793,2240,324,422,87,419,1665,1736,749,212,724,1814,714,1287,
459,800,90,2303,1111,723,1761,1683,577,1110,794,728,1288,2309,410,726,1103,
2425,1766,578,1721,370,843,1658,833,1685,1522,795,1098,371,586,2313,420,]


In [7]:
entries

[<Entry index=127 label="CH2O(9) + OH(15) <=> H2O(8) + HCO(19)">,
 <Entry index=180 label="CH(3) + H2(13) <=> CH2(23) + H(14)">,
 <Entry index=247 label="butane(1) + OH(15) <=> H2O(8) + SC4H9(183)">,
 <Entry index=288 label="butane(1) + OH(15) <=> H2O(8) + PC4H9(182)">,
 <Entry index=290 label="butane(1) + HO2(16) <=> H2O2(17) + PC4H9(182)">,
 <Entry index=294 label="CH2O(9) + PC4H9(182) <=> butane(1) + HCO(19)">,
 <Entry index=324 label="S(229) <=> S(225)">,
 <Entry index=325 label="S(225) <=> S(244)">,
 <Entry index=326 label="S(225) <=> S(238)">,
 <Entry index=370 label="CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)">,
 <Entry index=371 label="CH3CHO(35) + PC4H9(182) <=> butane(1) + CH2CHO(21)">,
 <Entry index=585 label="CH2(23) + C3H6(12) <=> CH3(18) + C3H5-A(94)">,
 <Entry index=591 label="CH3CHO(35) + C3H5-A(94) <=> C3H6(12) + CH3CO(20)">,
 <Entry index=650 label="HO2(16) + C4H8(748) <=> O2(2) + SC4H9(183)">,
 <Entry index=655 label="HO2(16) + C4H8(748) <=> O2(2) + PC4H9(182

In [8]:
# compile it all into a single database and a single library which I'll call harris_butane
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.libraries['kinetics'] = rmgpy.data.kinetics.KineticsLibrary()
ark_kinetics_database.libraries['kinetics'].label = 'harris_butane'
ark_kinetics_database.libraries['kinetics'].name = 'harris_butane'
ark_kinetics_database.libraries['kinetics'].entries = OrderedDict()
for entry in entries:
#     if entry.index not in top_50:
#         print(f'skipping non-top-50 {entry.index}')
#         continue
    ark_kinetics_database.libraries['kinetics'].entries[entry.label] = entry

In [9]:
# save the results
output_path = os.path.join(DFT_DIR, 'kinetics')
ark_kinetics_database.save_libraries(output_path, reindex=False)

In [10]:
print(len(ark_kinetics_database.libraries['kinetics'].entries), 'saved')

72 saved


In [11]:
for entry in ark_kinetics_database.libraries['kinetics'].entries:
    print(ark_kinetics_database.libraries['kinetics'].entries[entry].index, 
          ark_kinetics_database.libraries['kinetics'].entries[entry].label)

127 CH2O(9) + OH(15) <=> H2O(8) + HCO(19)
180 CH(3) + H2(13) <=> CH2(23) + H(14)
247 butane(1) + OH(15) <=> H2O(8) + SC4H9(183)
288 butane(1) + OH(15) <=> H2O(8) + PC4H9(182)
290 butane(1) + HO2(16) <=> H2O2(17) + PC4H9(182)
294 CH2O(9) + PC4H9(182) <=> butane(1) + HCO(19)
324 S(229) <=> S(225)
325 S(225) <=> S(244)
326 S(225) <=> S(238)
370 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)
371 CH3CHO(35) + PC4H9(182) <=> butane(1) + CH2CHO(21)
585 CH2(23) + C3H6(12) <=> CH3(18) + C3H5-A(94)
591 CH3CHO(35) + C3H5-A(94) <=> C3H6(12) + CH3CO(20)
650 HO2(16) + C4H8(748) <=> O2(2) + SC4H9(183)
655 HO2(16) + C4H8(748) <=> O2(2) + PC4H9(182)
739 C4H8(188) + CH2CHO(21) <=> CH3CHO(35) + C4H7(190)
748 HCO(19) + C4H8(189) <=> CH2O(9) + C4H7(190)
749 HCO(19) + C4H8(188) <=> CH2O(9) + C4H7(190)
754 C4H8(189) + CH2(23) <=> CH3(18) + C4H7(190)
894 C3H6(12) + S(244) <=> S(787) + C3H5-A(94)
985 HO2(16) + C2H4O(703) <=> O2(2) + C2H5O(49)
1003 HO2(16) + C4H7(191) <=> O2(2) + C4H8(188)
1026 CH3CHO(35) +